# ComfyUI Cloud Notebook

## Install Requirements

In [ ]:
#@title Environment Setup

import os

OPTIONS = {}

UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

# Set ComfyUI folder path
comfyui_folder = './ComfyUI'

# Check if ComfyUI folder exists, if not, clone the repository
if not os.path.exists(comfyui_folder):
    print("Cloning ComfyUI repository...")
    !git clone https://github.com/comfyanonymous/ComfyUI $comfyui_folder

if OPTIONS['UPDATE_COMFY_UI']:
    print("Updating ComfyUI...")
    !git -C $comfyui_folder pull

print("Installing dependencies...")
!pip install xformers!=0.0.18 -r $comfyui_folder/requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

# Installing additional requirements
!pip install numpy Pillow matplotlib lpips
!pip install git+https://github.com/openai/CLIP.git
!pip install aesthetic-predictor-v2-5

# Downloading model checkpoint
!wget -c https://huggingface.co/Comfy-Org/stable-diffusion-v1-5-archive/resolve/main/v1-5-pruned-emaonly-fp16.safetensors -P $comfyui_folder/models/checkpoints/

# Clip Vision
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors -O $comfyui_folder/models/clip_vision/CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/model.safetensors -O $comfyui_folder/models/clip_vision/CLIP-ViT-bigG-14-laion2B-39B-b160k.safetensors
!wget -c https://huggingface.co/Kwai-Kolors/Kolors-IP-Adapter-Plus/resolve/main/image_encoder/pytorch_model.bin -O $comfyui_folder/models/clip_vision/clip-vit-large-patch14-336.bin

# IP Adapter
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.safetensors -P $comfyui_folder/models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light_v11.bin -P $comfyui_folder/models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.safetensors -P $comfyui_folder/models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.safetensors -P $comfyui_folder/models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-full-face_sd15.safetensors -P $comfyui_folder/models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_vit-G.safetensors -P $comfyui_folder/models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors -P $comfyui_folder/models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors -P $comfyui_folder/models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus-face_sdxl_vit-h.safetensors -P $comfyui_folder/models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors -P $comfyui_folder/models/ipadapter/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15_light.safetensors -P $comfyui_folder/models/ipadapter/

# Cloning specified repositories into their corresponding folders inside custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git $comfyui_folder/custom_nodes/comfyui-manager
!git clone https://github.com/leon-etienne/ComfyUI_Scoring-Nodes.git $comfyui_folder/custom_nodes/ComfyUI_Scoring-Nodes
!git clone https://github.com/kijai/ComfyUI-KJNodes.git $comfyui_folder/custom_nodes/comfyui-kjnodes
!git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git $comfyui_folder/custom_nodes/comfyui-custom-scripts
!git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus.git $comfyui_folder/custom_nodes/comfyui_ipadapter_plus
!git clone https://github.com/MoonGoblinDev/Civicomfy.git $comfyui_folder/custom_nodes/civicomfy

## Run Server

In [2]:
# Download cloudflared
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# Python script to start the server and run cloudflared
import subprocess
import threading
import time
import socket
import os

# Set the ComfyUI folder path
comfyui_folder = './ComfyUI'

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

    p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')

# Start iframe thread to check if ComfyUI is running
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

# Start ComfyUI server from the directory above without using %cd
subprocess.run(["python", os.path.join(comfyui_folder, "main.py")])

--2025-04-29 12:33:52--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.4.0/cloudflared-linux-amd64.deb [following]
--2025-04-29 12:33:52--  https://github.com/cloudflare/cloudflared/releases/download/2025.4.0/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/d7e7703c-c0be-4512-b40f-145c402e03fd?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250429%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250429T123352Z&X-Amz-Expires=300&X-Amz-Signature=678983aac7e25baf8c1d42292d3e0df7b762279435582a73e1650e960194


Prestartup times for custom nodes:
   2.0 seconds: /home/jovyan/ComfyUI/custom_nodes/comfyui-manager

/opt/conda/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
Checkpoint files will always be loaded safely.
Total VRAM 19968 MB, total RAM 2051952 MB
pytorch version: 2.6.0+cu118
xformers version: 0.0.29.post3
Set vram state to: NORMAL_VRAM
Device: cuda:0 NVIDIA A100-SXM4-80GB MIG 2g.20gb : cudaMallocAsync
Using xformers attention
Python version: 3.11.10 | packaged by conda-forge | (main, Sep 30 2024, 18:08:57) [GCC 13.3.0]
ComfyUI version: 0.3.30
ComfyUI frontend version: 1.17.11
[Prompt Server] web root: /opt/conda/lib/python3.11/site-packages/comfyui_frontend_package/static
Total VRAM 19968 MB, total RAM 2051952 MB
pytorch version: 2.6

[Civicomfy Config] Ensured 'other' model directory exists: /home/jovyan/ComfyUI/models/custom_nodes/civicomfy/other_models
------------------------------
[Civicomfy Config Initialized]
  - Plugin Root: /home/jovyan/ComfyUI/custom_nodes/civicomfy
  - Web Directory: /home/jovyan/ComfyUI/custom_nodes/civicomfy/web
  - ComfyUI Models Dir: /home/jovyan/ComfyUI/models
  - 'Other' Models Subfolder (relative): custom_nodes/civicomfy/other_models
------------------------------
[Civicomfy Manager] Warning: ComfyUI folder_paths not available. Path validation/opening might be limited.
[Manager] History file not found (/home/jovyan/ComfyUI/custom_nodes/civicomfy/download_history.json). Starting with empty history.
Civitai Download Manager starting (Max Concurrent: 3).
[Manager] Process queue thread started.
[Civicomfy] Server routes registered.
[Civicomfy] Core modules imported successfully.
------------------------------
--- Civicomfy Custom Extension Loaded ---
- Serving frontend files from: /hom

[ComfyUI-Manager] default cache updated: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/model-list.json
[ComfyUI-Manager] default cache updated: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/extension-node-map.json
[ComfyUI-Manager] default cache updated: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/custom-node-list.json
[ComfyUI-Manager] default cache updated: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/github-stats.json
2025-04-29 12:34:02.858573: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-29 12:34:03.715741: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


FETCH ComfyRegistry Data: 5/83
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


...


Import times for custom nodes:
   0.0 seconds: /home/jovyan/ComfyUI/custom_nodes/websocket_image_save.py
   0.0 seconds: /home/jovyan/ComfyUI/custom_nodes/comfyui-custom-scripts
   0.0 seconds: /home/jovyan/ComfyUI/custom_nodes/comfyui_ipadapter_plus
   0.1 seconds: /home/jovyan/ComfyUI/custom_nodes/civicomfy
   0.1 seconds: /home/jovyan/ComfyUI/custom_nodes/comfyui-manager
   0.3 seconds: /home/jovyan/ComfyUI/custom_nodes/comfyui-kjnodes
   8.8 seconds: /home/jovyan/ComfyUI/custom_nodes/ComfyUI_Scoring-Nodes

Starting server

To see the GUI go to: http://127.0.0.1:8188



ComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)

FETCH ComfyRegistry Data: 15/83
This is the URL to access ComfyUI: https://shake-virgin-peninsula-closes.trycloudflare.com                                   |
FETCH ComfyRegistry Data: 20/83
FETCH ComfyRegistry Data: 25/83
FETCH ComfyRegistry Data: 30/83
FETCH ComfyRegistry Data: 35/83
FETCH ComfyRegistry Data: 40/83
FETCH ComfyRegistry Data: 45/83
FETCH ComfyRegistry Data: 50/83
FETCH ComfyRegistry Data: 55/83
FETCH ComfyRegistry Data: 60/83
FETCH ComfyRegistry Data: 65/83
FETCH ComfyRegistry Data: 70/83
FETCH ComfyRegistry Data: 75/83
FETCH ComfyRegistry Data: 80/83
FETCH ComfyRegistry Data [DONE]
FETCH DATA from: https://raw.githubusercontent.com/ltdrdata/ComfyUI-Manager/main/custom-node-list.json [DONE]


[ComfyUI-Manager] default cache updated: https://api.comfy.org/nodes
[ComfyUI-Manager] All startup tasks have been completed.

Stopped server
Exception ignored in atexit callback

[Manager] Shutdown requested.
[Manager] Requesting cancellation for 0 active and 0 queued downloads on shutdown...


KeyboardInterrupt: 